Спочатку підключимо потрібні біліотеки
---

In [46]:
import os
import requests
from datetime import datetime, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import timeit
import time
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import OneHotEncoder
print("Setup done")

Setup done


Завантаження та підготовка даних 
---

In [2]:

url = "https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip"
folder = "data"
if not os.path.exists("data"):
    os.makedirs("data")

filename = os.path.join(folder, f"household_power_consumption.zip")
try:
    recieved_data = requests.get(url, timeout=3)
    print(recieved_data)
    recieved_data.raise_for_status()
    with open(filename, "wb") as file:
        file.write(recieved_data.content)
    print(f"Saving file in {filename} [OK]")

    if zipfile.is_zipfile(filename):
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall("data")

    os.remove(os.path.join(filename))
    dataframe = pd.DataFrame()

    file = os.listdir("data") 
    dataframe = pd.read_csv(os.path.join(folder, file[0]), sep=';', na_values='?')
    dataframe = dataframe.dropna()
    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format='%d/%m/%Y')
    dataframe['Time'] = pd.to_datetime(dataframe['Time'], format='%H:%M:%S').dt.time

    dataframe_numpy = dataframe.to_numpy()
    display(dataframe)

except Exception as e:
    print(f"[Error] {e}")

<Response [200]>
Saving file in data\household_power_consumption.zip [OK]


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...,...,...
2075254,2010-11-26,20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2075255,2010-11-26,20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2075256,2010-11-26,21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0
2075257,2010-11-26,21:01:00,0.934,0.000,239.70,3.8,0.0,0.0,0.0


Завдання першого рівня
===

#### 1. Обрати всі записи, у яких загальна активна споживана потужність перевищує 5 кВт.

Pandas Dataframe

In [9]:
edited = dataframe[dataframe["Global_active_power"] > 5]
display(edited)
time_100 = timeit.timeit('dataframe[dataframe["Global_active_power"] > 5]', globals = globals(), number=100)
print(f"Time is {time_100:.6f} seconds")

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
11,2006-12-16,17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
12,2006-12-16,17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0
...,...,...,...,...,...,...,...,...,...
2069356,2010-11-22,18:40:00,5.408,0.150,231.50,23.6,48.0,0.0,0.0
2069357,2010-11-22,18:41:00,5.528,0.144,232.48,24.6,53.0,0.0,0.0
2071586,2010-11-24,07:50:00,5.172,0.050,235.18,22.0,0.0,38.0,17.0
2071587,2010-11-24,07:51:00,5.750,0.000,234.40,24.6,0.0,39.0,17.0


Time is 0.134999 seconds


Numpy Array

In [10]:
index = dataframe.columns.get_loc('Global_active_power')
filtered = dataframe_numpy[dataframe_numpy[:, index] > 5]
display(filtered)

time_100 = timeit.timeit("dataframe_numpy[dataframe_numpy[:, dataframe.columns.get_loc('Global_active_power')] > 5]", globals = globals(), number=100)

print(f"Time is {time_100:.6f} seconds")

array([[Timestamp('2006-12-16 00:00:00'), datetime.time(17, 25), 5.36,
        ..., 0.0, 1.0, 16.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 26), 5.374,
        ..., 0.0, 2.0, 17.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 27), 5.388,
        ..., 0.0, 1.0, 17.0],
       ...,
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 50), 5.172,
        ..., 0.0, 38.0, 17.0],
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 51), 5.75,
        ..., 0.0, 39.0, 17.0],
       [Timestamp('2010-11-25 00:00:00'), datetime.time(7, 21), 5.074,
        ..., 1.0, 2.0, 18.0]], shape=(17547, 9), dtype=object)

Time is 2.494909 seconds


#### 2. Обрати всі записи, у яких вольтаж перевищую 235 В.

Pandas Dataframe

In [12]:
edited = dataframe[dataframe['Voltage'] > 235]
display(edited)
time_100 = timeit.timeit("dataframe[dataframe['Voltage'] > 235]", globals = globals(), number=100)
print(f"Time is {time_100:.6f} seconds")

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
5,2006-12-16,17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
6,2006-12-16,17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
7,2006-12-16,17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
14,2006-12-16,17:38:00,4.054,0.422,235.24,17.6,0.0,1.0,17.0
...,...,...,...,...,...,...,...,...,...
2075254,2010-11-26,20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2075255,2010-11-26,20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2075256,2010-11-26,21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0
2075257,2010-11-26,21:01:00,0.934,0.000,239.70,3.8,0.0,0.0,0.0


Time is 4.862360 seconds


Numpy Array

In [13]:
index = dataframe.columns.get_loc('Voltage')
filtered = dataframe_numpy[dataframe_numpy[:, index] > 235]
display(filtered)

time_100 = timeit.timeit("dataframe_numpy[dataframe_numpy[:, dataframe.columns.get_loc('Voltage')] > 235]", globals = globals(), number=100)

print(f"Time is {time_100:.6f} seconds")

array([[Timestamp('2006-12-16 00:00:00'), datetime.time(17, 28), 3.666,
        ..., 0.0, 1.0, 17.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 29), 3.52,
        ..., 0.0, 2.0, 17.0],
       [Timestamp('2006-12-16 00:00:00'), datetime.time(17, 30), 3.702,
        ..., 0.0, 1.0, 17.0],
       ...,
       [Timestamp('2010-11-26 00:00:00'), datetime.time(21, 0), 0.938,
        ..., 0.0, 0.0, 0.0],
       [Timestamp('2010-11-26 00:00:00'), datetime.time(21, 1), 0.934,
        ..., 0.0, 0.0, 0.0],
       [Timestamp('2010-11-26 00:00:00'), datetime.time(21, 2), 0.932,
        ..., 0.0, 0.0, 0.0]], shape=(1952491, 9), dtype=object)

Time is 12.867669 seconds


#### 3. Обрати всі записи, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

Pandas Dataframe

In [22]:
edited = dataframe[dataframe["Global_intensity"].between(19, 20)]
edited = edited[edited["Sub_metering_2"] > edited["Sub_metering_3"]]
display(edited)

time_100 = timeit.timeit("dataframe[(dataframe['Global_intensity'].between(19, 20)) & (edited['Sub_metering_2'] > edited['Sub_metering_3'])]", globals = globals(), number=100)
print(f"Time is {time_100:.6f} seconds")

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
45,2006-12-16,18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0
460,2006-12-17,01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0
464,2006-12-17,01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0
475,2006-12-17,01:19:00,4.636,0.140,237.37,19.4,0.0,36.0,0.0
476,2006-12-17,01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0
...,...,...,...,...,...,...,...,...,...
2071589,2010-11-24,07:53:00,4.666,0.000,235.72,19.8,0.0,39.0,17.0
2071590,2010-11-24,07:54:00,4.694,0.000,236.78,19.8,0.0,39.0,18.0
2071591,2010-11-24,07:55:00,4.602,0.000,237.08,19.4,0.0,40.0,17.0
2071592,2010-11-24,07:56:00,4.536,0.000,237.03,19.0,0.0,39.0,17.0


Time is 12.345484 seconds


Numpy Array

In [ ]:
gi_idx = dataframe.columns.get_loc('Global_intensity')
sub2_idx = dataframe.columns.get_loc('Sub_metering_2')
sub3_idx = dataframe.columns.get_loc('Sub_metering_3')
filtered = dataframe_numpy[(dataframe_numpy[:, gi_idx] >= 19) & (dataframe_numpy[:, gi_idx] <= 20) & (dataframe_numpy[:, sub2_idx] > dataframe_numpy[:, sub3_idx])]
display(filtered)
display(pd.DataFrame(filtered, columns=dataframe.columns))

time_100 = timeit.timeit("dataframe_numpy[(dataframe_numpy[:, gi_idx] >= 19) & (dataframe_numpy[:, dataframe.columns.get_loc('Global_intensity')] <= 20) &"
"(dataframe_numpy[:, dataframe.columns.get_loc('Sub_metering_2')] > dataframe_numpy[:, dataframe.columns.get_loc('Sub_metering_3')])]",
                         globals = globals(), number=100)

print(f"Time is {time_100:.6f} seconds")

array([[Timestamp('2006-12-16 00:00:00'), datetime.time(18, 9), 4.464,
        ..., 0.0, 37.0, 16.0],
       [Timestamp('2006-12-17 00:00:00'), datetime.time(1, 4), 4.582,
        ..., 0.0, 13.0, 0.0],
       [Timestamp('2006-12-17 00:00:00'), datetime.time(1, 8), 4.618,
        ..., 0.0, 27.0, 0.0],
       ...,
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 55), 4.602,
        ..., 0.0, 40.0, 17.0],
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 56), 4.536,
        ..., 0.0, 39.0, 17.0],
       [Timestamp('2010-11-24 00:00:00'), datetime.time(7, 57), 4.626,
        ..., 0.0, 39.0, 17.0]], shape=(2509, 9), dtype=object)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0
1,2006-12-17,01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0
2,2006-12-17,01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0
3,2006-12-17,01:19:00,4.636,0.14,237.37,19.4,0.0,36.0,0.0
4,2006-12-17,01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0
...,...,...,...,...,...,...,...,...,...
2504,2010-11-24,07:53:00,4.666,0.0,235.72,19.8,0.0,39.0,17.0
2505,2010-11-24,07:54:00,4.694,0.0,236.78,19.8,0.0,39.0,18.0
2506,2010-11-24,07:55:00,4.602,0.0,237.08,19.4,0.0,40.0,17.0
2507,2010-11-24,07:56:00,4.536,0.0,237.03,19.0,0.0,39.0,17.0


Time is 5.871257 seconds


#### 4. Обрати випадковим чином 500000 записів (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії

Pandas Dataframe

In [34]:
part = dataframe.sample(n=500000, replace=False, random_state=78)
mean_sub1_pd = part['Sub_metering_1'].mean()
mean_sub2_pd = part['Sub_metering_2'].mean()
mean_sub3_pd = part['Sub_metering_3'].mean()

print("Pandas Sample:")
display(part)
print("Середнє sub_metering_1:", mean_sub1_pd)
print("Середнє sub_metering_2:", mean_sub2_pd)
print("Середнє sub_metering_3:", mean_sub3_pd)

time_100 = timeit.timeit("dataframe.sample(n=500000, replace=False, random_state=78)[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()", globals = globals(), number=100)
print(f"Time is {time_100:.6f} seconds")

Pandas Sample:


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1937879,2010-08-23,11:23:00,0.270,0.104,241.48,1.2,0.0,0.0,0.0
765688,2008-05-31,10:52:00,0.568,0.212,239.36,2.8,0.0,0.0,1.0
1070189,2008-12-28,21:53:00,0.284,0.080,242.27,1.2,0.0,1.0,0.0
1036014,2008-12-05,04:18:00,0.272,0.054,243.77,1.2,0.0,0.0,0.0
905385,2008-09-05,11:09:00,0.142,0.000,238.92,0.6,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
116076,2007-03-07,08:00:00,1.350,0.084,236.98,5.6,0.0,0.0,17.0
174325,2007-04-16,18:49:00,0.126,0.000,242.03,0.6,0.0,0.0,0.0
60060,2007-01-27,10:24:00,1.612,0.234,240.13,6.8,0.0,0.0,17.0
1209194,2009-04-04,10:38:00,1.478,0.066,242.85,6.0,0.0,0.0,18.0


Середнє sub_metering_1: 1.137488
Середнє sub_metering_2: 1.30552
Середнє sub_metering_3: 6.47861
Time is 7.690747 seconds


Numpy Array

In [39]:
def numpy_random_func(dataframe_numpy):
    sample_np = dataframe_numpy[np.random.choice(dataframe_numpy.shape[0], size=500000, replace=False)]
    mean_sub1 = np.mean(sample_np[:, dataframe.columns.get_loc('Sub_metering_1')])
    mean_sub2 = np.mean(sample_np[:, dataframe.columns.get_loc('Sub_metering_2')])
    mean_sub3 = np.mean(sample_np[:, dataframe.columns.get_loc('Sub_metering_3')])

    return mean_sub1, mean_sub2, mean_sub3, sample_np

mean1, mean2, mean3, filtered = numpy_random_func(dataframe_numpy)
display(filtered)
display(pd.DataFrame(filtered, columns=dataframe.columns))
print(f"Sub_metering_1: {mean1}")
print(f"Sub_metering_2: {mean2}")
print(f"Sub_metering_3: {mean3}")

time_100 = timeit.timeit("numpy_random_func(dataframe_numpy)", globals = globals(), number=100)

print(f"Time is {time_100:.6f} seconds")

array([[Timestamp('2009-07-06 00:00:00'), datetime.time(14, 41), 0.226,
        ..., 0.0, 1.0, 1.0],
       [Timestamp('2008-07-22 00:00:00'), datetime.time(17, 15), 1.208,
        ..., 0.0, 0.0, 19.0],
       [Timestamp('2008-10-13 00:00:00'), datetime.time(19, 42), 1.494,
        ..., 0.0, 1.0, 0.0],
       ...,
       [Timestamp('2010-01-08 00:00:00'), datetime.time(4, 2), 0.256,
        ..., 0.0, 0.0, 0.0],
       [Timestamp('2010-09-22 00:00:00'), datetime.time(12, 45), 2.128,
        ..., 0.0, 1.0, 1.0],
       [Timestamp('2009-02-15 00:00:00'), datetime.time(9, 23), 2.49,
        ..., 0.0, 0.0, 19.0]], shape=(500000, 9), dtype=object)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2009-07-06,14:41:00,0.226,0.118,241.63,1.0,0.0,1.0,1.0
1,2008-07-22,17:15:00,1.208,0.0,239.49,5.0,0.0,0.0,19.0
2,2008-10-13,19:42:00,1.494,0.346,237.89,6.4,0.0,1.0,0.0
3,2008-12-01,09:05:00,1.934,0.048,239.58,8.0,0.0,0.0,17.0
4,2009-01-01,11:15:00,0.556,0.072,240.39,2.6,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
499995,2009-07-25,06:25:00,0.232,0.146,243.94,1.0,0.0,0.0,1.0
499996,2009-04-21,15:44:00,0.298,0.0,242.18,1.2,0.0,0.0,1.0
499997,2010-01-08,04:02:00,0.256,0.0,249.59,1.0,0.0,0.0,0.0
499998,2010-09-22,12:45:00,2.128,0.652,239.44,9.4,0.0,1.0,1.0


Sub_metering_1: 1.128218
Sub_metering_2: 1.293056
Sub_metering_3: 6.460632
Time is 17.005307 seconds


#### 5 Обрати ті записи, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

Pandas Dataframe

In [53]:
def fifth_func(dataframe):
    filtered = dataframe[(dataframe['Time'] > pd.to_datetime('18:00:00').time()) & (dataframe['Global_active_power'] > 6)]
    filtered = filtered[(filtered['Sub_metering_2'] > filtered['Sub_metering_1']) & (filtered['Sub_metering_2'] > filtered['Sub_metering_3'])]
    part1 = filtered.iloc[:len(filtered) // 2].iloc[2::3]
    part2 = filtered.iloc[len(filtered) // 2:].iloc[3::4]
    filtered = pd.concat([part1, part2])
    return filtered

filtered = fifth_func(dataframe)
display(filtered)

time_100 = timeit.timeit("fifth_func(dataframe)", globals = globals(), number=100)
print(f"Time is {time_100:.6f} seconds")

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
43,2006-12-16,18:07:00,6.474,0.144,231.85,27.8,0.0,37.0,16.0
3007,2006-12-18,19:31:00,6.158,0.442,229.08,27.0,0.0,36.0,0.0
17497,2006-12-28,21:01:00,7.062,0.270,235.76,30.2,2.0,65.0,17.0
17500,2006-12-28,21:04:00,7.376,0.238,234.67,31.4,1.0,72.0,17.0
17503,2006-12-28,21:07:00,7.248,0.000,235.34,30.8,1.0,72.0,17.0
...,...,...,...,...,...,...,...,...,...
2066466,2010-11-20,18:30:00,6.620,0.314,229.00,29.2,18.0,34.0,16.0
2066470,2010-11-20,18:34:00,6.266,0.360,229.82,27.6,14.0,35.0,16.0
2066474,2010-11-20,18:38:00,6.302,0.376,229.65,27.8,15.0,34.0,17.0
2066478,2010-11-20,18:42:00,6.238,0.344,229.37,27.6,14.0,35.0,16.0


Time is 4.553877 seconds


Numpy Array

In [55]:
def fifth_func_(dataframe_numpy):
    filtered = dataframe_numpy[(dataframe_numpy[:, 1] > pd.to_datetime('18:00:00').time()) & (dataframe_numpy[:, 2] > 6) &
        (dataframe_numpy[:, 7] > dataframe_numpy[:, 6]) & (dataframe_numpy[:, 7] > dataframe_numpy[:, 8])]

    part1 = filtered[:len(filtered)//2][2::3]
    part2 = filtered[len(filtered)//2:][3::4]
    filtered = np.concatenate((part1, part2), axis=0)

    return filtered

filtered = fifth_func_(dataframe_numpy)
display(filtered)
display(pd.DataFrame(filtered, columns=dataframe.columns))

time_100 = timeit.timeit("fifth_func_(dataframe_numpy)", globals = globals(), number=100)
print(f"Time is {time_100:.6f} seconds")

array([[Timestamp('2006-12-16 00:00:00'), datetime.time(18, 7), 6.474,
        ..., 0.0, 37.0, 16.0],
       [Timestamp('2006-12-18 00:00:00'), datetime.time(19, 31), 6.158,
        ..., 0.0, 36.0, 0.0],
       [Timestamp('2006-12-28 00:00:00'), datetime.time(21, 1), 7.062,
        ..., 2.0, 65.0, 17.0],
       ...,
       [Timestamp('2010-11-20 00:00:00'), datetime.time(18, 38), 6.302,
        ..., 15.0, 34.0, 17.0],
       [Timestamp('2010-11-20 00:00:00'), datetime.time(18, 42), 6.238,
        ..., 14.0, 35.0, 16.0],
       [Timestamp('2010-11-20 00:00:00'), datetime.time(18, 46), 6.438,
        ..., 13.0, 39.0, 16.0]], shape=(308, 9), dtype=object)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,18:07:00,6.474,0.144,231.85,27.8,0.0,37.0,16.0
1,2006-12-18,19:31:00,6.158,0.442,229.08,27.0,0.0,36.0,0.0
2,2006-12-28,21:01:00,7.062,0.27,235.76,30.2,2.0,65.0,17.0
3,2006-12-28,21:04:00,7.376,0.238,234.67,31.4,1.0,72.0,17.0
4,2006-12-28,21:07:00,7.248,0.0,235.34,30.8,1.0,72.0,17.0
...,...,...,...,...,...,...,...,...,...
303,2010-11-20,18:30:00,6.62,0.314,229.0,29.2,18.0,34.0,16.0
304,2010-11-20,18:34:00,6.266,0.36,229.82,27.6,14.0,35.0,16.0
305,2010-11-20,18:38:00,6.302,0.376,229.65,27.8,15.0,34.0,17.0
306,2010-11-20,18:42:00,6.238,0.344,229.37,27.6,14.0,35.0,16.0


Time is 6.642904 seconds
